In [5]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

KT_vars = ['x','y','z1','z2','z3','z4','z5','v1','v2','v3','v4','v5']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m','l1','l2','l3','l4','l5','w1','w2','w3','w4','w5']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


x,y,z1,z2,z3,z4,z5,v1,v2,v3,v4,v5=KT_ring.gens()
t,m,l1,l2,l3,l4,l5,w1,w2,w3,w4,w5= coho_ring.gens()
b=var('b')
'''   w=SR.var('w',5)   '''
v=[v1,v2,v3,v4,v5]
w=[w1,w2,w3,w4,w5]
L=var('l')
z=[z1,z2,z3,z4,z5]


def character(p,x=x,y=y):
    return sum(y^(-r) * x^(-c) for r, c in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def dual(f):
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def tangent(p,x=x,y=y):
    total=0
    for r,c in p.cells():
        arm=p[r]-c-1
        leg=p.conjugate()[c]-r-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)


def structure_sheaf(p,x=x,y=y):
    return character_vb(p, (0,0),x,y)

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def measure_unsymmetrized(f, inv=True):
    """
    computes 1/(1-x^-1) for K-thy class x. If inv=false, returns 1/(1-x)
    """
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom



def degn(f,n,b=b):
    g=f.taylor(b,0,n+2)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

    

In [16]:
def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    '''gives list of fixed points of Hilb^n1 x Hilb^n2 x ... x Hilb^nN, input (n1,n2,...,nN)'''
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[Partitions(ptn[i]).list()]
        total+=[p for p in itertools.product(*lst)]
    return total

def Nvir(Z):
    '''virtual normal bundle of quot scheme at (Z_1,...,Z_N)'''
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            total+=(1-character(Z[i],x=x^-1,y=y^-1)*(1-x)*(1-y))*character(Z[j],x=x,y=y)*(z[j]/z[i])
    return total


def total_chern_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((1+r) for r in roots)
    return total

def top_chern_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((r) for r in roots)
    return total

def Chern_Num_quot(n,N,rank):
    total=0
    print(len(fixed_locus(n,N)))
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_chern_quot(Z,rank)
        total+=add
        print("added")
    return total

def DT_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))
        total+=add
    return total

def total_segre_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod(1/(1+r) for r in roots)
    return total

def Segre_Num_quot(n,N,rank):
    total=0
    print(len(fixed_locus(n,N)))
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z,rank)
        total+=add
        print("added")
    return total
def Segre_Num_quot_sym(n,N,rank):
    total=0
    print(len(fixed_locus(n,rank)))
    for Z in fixed_locus(n,rank):
        add=symmetrize(top_chern(-Nvir(Z)))*symmetrize(total_segre_quot(Z,N))
        total+=add
        print("added")
    return total


def determinant_quot(Z,n,rank):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,y)*(z[i])
        det *= determinant(Q)
    return (det^rank)*((prod(v[i] for i in range(rank)))^n)

def Verlinde_Num_quot(n,N,rank):
    total=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot(Z,n,rank)
        total+=add
    return total

def Verlinde_Num_quot_inv(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot(Z,n,rank))
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total


def deg0(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),0)

def deg02(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v),0))

def set_exp(f):
    for i in range(len(KT_vars)):
        f=SR(f).subs(KT_ring.gens()[i]==exp(coho_ring.gens()[i]))
    return f
def simplify_weight(f):
    return f(l1=0,l2=l1,l3=2*l1,l4=3*l1,l5=4*l5)

def symmetrize(f):
    if f in ZZ:
        return f
    return f(l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5)


def symmetrizeK(f):
    if f in ZZ:
        return f
    return f(z1=v1,z2=v2,z3=v3,z4=v4,z5=v5,v1=z1,v2=z2,v3=z3,v4=z4,v5=z5)

def all_check(start,finish):
    with open("symmetry2d-segre-checked.txt") as f:
        lines = f.read().splitlines()
    if len(lines)==0:
        n,N,r=[0,0,0]
    else:
        last_checked=lines[len(lines)-1]
        n,N,r=[int(a) for a in last_checked.split(",")]
    for i in range(start,finish+1):
        for j in range(1,2):
            k=j
            n,N,r=[i,j,k]
            if k<j:
                continue
            if j<=k:
                print("checking segre sym: n="+str(n)+", N="+str(N)+", r="+str(r))
                
                S1=Segre_Num_quot(n,N,r)
                S2=Segre_Num_quot_sym(n,N,r)
                print("comparing")
                diff=S1*(-1)^(n*N)-S2*(-1)^(n*r)
                if diff==0:
                    writeline("symmetry2d-segre-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                              
        print("all checked for: n="+str(i))
                        
                    
def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
                
                
def clear_all_files():
    open('SV2d-checked.txt', 'w').close()
    open('symmetry2d-segre-checked.txt', 'w').close()
    open('symmetry2d-Verlinde-checked.txt', 'w').close()
    open('verlinde2d-out.txt', 'w').close()
    open('segre2d-out.txt', 'w').close()
    open('SV2d-out.txt', 'w').close()
            

def check_nek(N,nlimit):
    with open("nek2d-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        print("checking n="+str(n)+", N="+str(N))
        if str(n)+','+str(N) in lines:
            print("n="+str(n)+", N="+str(N)+" already checked")
        else:
            a=Chern_Num_quot(n,N,N-1)
            b=(m+t)^n/factorial(n)/(m*t)^n
            if a==b:
                writeline("nek2d-checked.txt",str(n)+','+str(N))
                print("checked")
            else:
                print("failed")
                
def check_vanish(N,nlimit):
    with open("chern-vanish-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        for r in range(0,N-1):
            print("checking n="+str(n)+", N="+str(N)+', r='+str(r))
            if str(n)+','+str(N)+','+str(r) in lines:
                print("n="+str(n)+", N="+str(N)+','+str(r)+" already checked")
            else:
                a=Chern_Num_quot(n,N,r)
                if a==0:
                    writeline("chern-vanish-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    
def check_vanish2(N,nlimit):
    with open("verlinde-vanish-checked.txt") as f:
        lines = f.read().splitlines()
    for n in range(1,nlimit+1):
        for r in range(1,N):
            print("checking n="+str(n)+", N="+str(N)+', r='+str(r))
            if str(n)+','+str(N)+','+str(r) in lines:
                print("n="+str(n)+", N="+str(N)+','+str(r)+" already checked")
            else:
                a=Verlinde_Num_quot_inv(n,N,r)
                if a==0:
                    writeline("verlinde-vanish-checked.txt",str(n)+','+str(N)+','+str(r))
                    print("checked")
                else:
                    print("failed")

In [18]:
all_check(2,2)

checking segre sym: n=2, N=1, r=1
2
added
added
2
added
added
comparing
checked
checking segre sym: n=2, N=2, r=2
5
added
added
added
added
added
5
added
added
added
added
added
comparing
checked
checking segre sym: n=2, N=3, r=3
9
added
added
added
added
added
added
added
added
added
9
added
added
added
added
added
added
added
added
added
comparing
checked
all checked for: n=2


In [37]:
check_vanish2(2,8)

checking n=1, N=2, r=1
n=1, N=2,1 already checked
checking n=2, N=2, r=1
n=2, N=2,1 already checked
checking n=3, N=2, r=1
n=3, N=2,1 already checked
checking n=4, N=2, r=1
n=4, N=2,1 already checked
checking n=5, N=2, r=1
n=5, N=2,1 already checked
checking n=6, N=2, r=1
n=6, N=2,1 already checked
checking n=7, N=2, r=1
n=7, N=2,1 already checked
checking n=8, N=2, r=1
added1/185
added2/185
added3/185
added4/185
added5/185
added6/185
added7/185
added8/185
added9/185
added10/185
added11/185
added12/185
added13/185
added14/185
added15/185
added16/185
added17/185
added18/185
added19/185
added20/185
added21/185
added22/185
added23/185
added24/185
added25/185
added26/185
added27/185
added28/185
added29/185
added30/185
added31/185
added32/185
added33/185
added34/185
added35/185
added36/185
added37/185
added38/185
added39/185
added40/185
added41/185
added42/185
added43/185
added44/185
added45/185
added46/185
added47/185
added48/185
added49/185
added50/185
added51/185
added52/185
added53/185


In [38]:
check_vanish2(3,5)
check_vanish2(4,3)
check_vanish2(5,2)

checking n=1, N=3, r=1
added1/3
added2/3
added3/3
checked
checking n=1, N=3, r=2
added1/3
added2/3
added3/3
checked
checking n=2, N=3, r=1
added1/9
added2/9
added3/9
added4/9
added5/9
added6/9
added7/9
added8/9
added9/9
checked
checking n=2, N=3, r=2
added1/9
added2/9
added3/9
added4/9
added5/9
added6/9
added7/9
added8/9
added9/9
checked
checking n=3, N=3, r=1
added1/22
added2/22
added3/22
added4/22
added5/22
added6/22
added7/22
added8/22
added9/22
added10/22
added11/22
added12/22
added13/22
added14/22
added15/22
added16/22
added17/22
added18/22
added19/22
added20/22
added21/22
added22/22
checked
checking n=3, N=3, r=2
added1/22
added2/22
added3/22
added4/22
added5/22
added6/22
added7/22
added8/22
added9/22
added10/22
added11/22
added12/22
added13/22
added14/22
added15/22
added16/22
added17/22
added18/22
added19/22
added20/22
added21/22
added22/22
checked
checking n=4, N=3, r=1
added1/51
added2/51
added3/51
added4/51
added5/51
added6/51
added7/51
added8/51
added9/51
added10/51
added11/

In [39]:
check_vanish2(5,3)

checking n=1, N=5, r=1
n=1, N=5,1 already checked
checking n=1, N=5, r=2
n=1, N=5,2 already checked
checking n=1, N=5, r=3
n=1, N=5,3 already checked
checking n=1, N=5, r=4
n=1, N=5,4 already checked
checking n=2, N=5, r=1
n=2, N=5,1 already checked
checking n=2, N=5, r=2
n=2, N=5,2 already checked
checking n=2, N=5, r=3
n=2, N=5,3 already checked
checking n=2, N=5, r=4
n=2, N=5,4 already checked
checking n=3, N=5, r=1
added1/65
added2/65
added3/65
added4/65
added5/65
added6/65
added7/65
added8/65
added9/65
added10/65
added11/65
added12/65
added13/65


KeyboardInterrupt: 

In [ ]:
check_vanish(5,3)

In [9]:
check_vanish(3,5)

checking n=1, N=3, r=0
n=1, N=3,0 already checked
checking n=1, N=3, r=1
n=1, N=3,1 already checked
checking n=2, N=3, r=0
n=2, N=3,0 already checked
checking n=2, N=3, r=1
n=2, N=3,1 already checked
checking n=3, N=3, r=0
n=3, N=3,0 already checked
checking n=3, N=3, r=1
n=3, N=3,1 already checked
checking n=4, N=3, r=0
n=4, N=3,0 already checked
checking n=4, N=3, r=1
n=4, N=3,1 already checked
checking n=5, N=3, r=0
n=5, N=3,0 already checked
checking n=5, N=3, r=1
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added

In [10]:
check_vanish(4,3)

checking n=1, N=4, r=0
added
added
added
added
checked
checking n=1, N=4, r=1
added
added
added
added
checked
checking n=1, N=4, r=2
added
added
added
added
checked
checking n=2, N=4, r=0
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=2, N=4, r=1
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=2, N=4, r=2
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=3, N=4, r=0
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
checked
checking n=3, N=4, r=1
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
added
adde

In [17]:
check_vanish(5,2)

checking n=1, N=5, r=0
n=1, N=5,0 already checked
checking n=1, N=5, r=1
n=1, N=5,1 already checked
checking n=1, N=5, r=2
n=1, N=5,2 already checked
checking n=1, N=5, r=3
n=1, N=5,3 already checked
checking n=2, N=5, r=0
n=2, N=5,0 already checked
checking n=2, N=5, r=1
n=2, N=5,1 already checked
checking n=2, N=5, r=2
n=2, N=5,2 already checked
checking n=2, N=5, r=3
n=2, N=5,3 already checked
